# Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import math
import urllib.request as ul 
from bs4 import BeautifulSoup

In [2]:
%matplotlib inline

In [3]:
import csv

In [5]:
df = pd.read_csv("listings.csv")

# Webscrapping to get the table of the boroughs, neighborhoods and zip codes

In [6]:
url = "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm"

url_response=ul.urlopen(url, timeout=5)
soup = BeautifulSoup(url_response)

#table we are looking into
table = soup.find_all('table')[0]

#borough: list of boroughs, neighborhood: list of neighborhoods, zip_code: list of zip codes
borough = list()
neighborhood = list()
zip_code = list()
zip_code_unique = list()
length = list()

#patterns for regular expressions - found printing table.find_all('tr')
pattern_borough = '"header1.+>(.+)</td>' 
pattern_neighborhood = '"header2">\s(.+)</td>'
pattern_zip = '"header3">\s(.+)</td>'

for line in table.find_all('td'):
    regex_borough = re.compile(pattern_borough) 
    borough.extend(regex_borough.findall(str(line)))
    
    regex_neighborhood = re.compile(pattern_neighborhood) 
    neighborhood.extend(regex_neighborhood.findall(str(line)))
    
    if len(neighborhood) > len(borough):
        borough.append(borough[-1])
        
    regex_zip = re.compile(pattern_zip) 
    zip_code.extend(regex_zip.findall(str(line)))  
    
#print(borough)
#print(neighborhood)
#print(zip_code)

#code to split the zip codes
for i in range(len(zip_code)):
    regex_zip2 = re.compile('(\s*[,]\s*)')
    zip_ext = regex_zip.sub(',', zip_code[i]).split(',')
    
    #list of unique zip codes
    zip_code_unique.extend(zip_ext)
    #number of split realized to get the corresponding borough and neighborhood
    length.append(len(zip_ext))
    
#code to duplicate the borough and neighborhood so that one given index gives both information
#must have a reverse loop to insert at the appropriate location
for i in range(len(borough))[::-1]:
    j = 0
    #j is a counter to insert the same borough/neighborhood at the given index
    while length[i] - j > 1:
        borough.insert(i, borough[i])
        neighborhood.insert(i, neighborhood[i])
        j += 1

#Test to compare with the table
i = 67
print(borough[i])
print(neighborhood[i])
print(zip_code_unique[i])

#print(len(borough))
#print(len(neighborhood))
#print(len(zip_code_unique))
#178 rows: sanity check ok done with Excel
    

Manhattan
Chelsea and Clinton
10001


In [7]:
#cleaning the zip_code_unique list to avoid " "
#print(zip_code_unique)
for i in range(len(zip_code_unique)):
    if len(zip_code_unique[i]) == 6:
        zip_code_unique[i] = zip_code_unique[i][1:6]
        
print(zip_code_unique)

['10453', '10457', '10460', '10458', '10467', '10468', '10451', '10452', '10456', '10454', '10455', '10459', '10474', '10463', '10471', '10466', '10469', '10470', '10475', '10461', '10462', '10464', '10465', '10472', '10473', '11212', '11213', '11216', '11233', '11238', '11209', '11214', '11228', '11204', '11218', '11219', '11230', '11234', '11236', '11239', '11223', '11224', '11229', '11235', '11201', '11205', '11215', '11217', '11231', '11203', '11210', '11225', '11226', '11207', '11208', '11211', '11222', '11220', '11232', '11206', '11221', '11237', '10026', '10027', '10030', '10037', '10039', '10001', '10011', '10018', '10019', '10020', '10036', '10029', '10035', '10010', '10016', '10017', '10022', '10012', '10013', '10014', '10004', '10005', '10006', '10007', '10038', '10280', '10002', '10003', '10009', '10021', '10028', '10044', '10065', '10075', '10128', '10023', '10024', '10025', '10031', '10032', '10033', '10034', '10040', '11361', '11362', '11363', '11364', '11354', '11355', 

# Raw data

In [8]:
data0 = pd.read_csv('listings.csv')

In [9]:
# list of all variables
data0.columns.values

array(['id', 'scrape_id', 'last_scraped', 'name', 'picture_url', 'host_id',
       'host_name', 'host_since', 'host_picture_url', 'street',
       'neighbourhood', 'neighbourhood_cleansed', 'city', 'state',
       'zipcode', 'market', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'square_feet', 'price',
       'weekly_price', 'monthly_price', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'calendar_updated',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'host_listing_count'], dtype=object)

In [10]:
nrow = len(data0.index)
nrow

27392

# Select relevant variables

In [11]:
data = data0[['name', 'neighbourhood_cleansed', 'city', 'zipcode', 'latitude', 'longitude', 'is_location_exact',
          'property_type', 'room_type', 'accommodates','bathrooms', 'bedrooms', 'beds', 'square_feet', 'price',
           'weekly_price', 'monthly_price', 'review_scores_value']]

In [12]:
data.head(2)

,name,neighbourhood_cleansed,city,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,square_feet,price,weekly_price,monthly_price,review_scores_value
0,Stay like a real New Yorker!,Midtown East,New York,10022,40.756852,-73.964754,t,Apartment,Entire home/apt,2,1,1,1,NaN,$160.00,"$1,000.00",NaN,9
1,Apartment 20 Minutes Times Square,Hamilton Heights,New York,NaN,40.830599,-73.941014,f,Apartment,Entire home/apt,10,1,3,3,NaN,$105.00,NaN,NaN,8


# Cleaning data

## Prices : convert into right format, replace missing values

In [13]:
day_price = []
for i in range(nrow):
    # remove commas ans $ sign, convert into float
    day_price.append((float(data.price[i].replace(',','')[1:])))

    
# there are null values for weekly / monthly prices
# use daily price to fill missing values

week_price, month_price = [], []    
for i in range(nrow):
    if pd.isnull(data.weekly_price[i]) == True :
        week_price.append(7*day_price[i])
    else :
        week_price.append(float(data.weekly_price[i].replace(',','')[1:]))
        
for i in range(nrow):
    if pd.isnull(data.monthly_price[i]) == True :
        month_price.append(30*day_price[i])
    else :
        month_price.append(float(data.monthly_price[i].replace(',','')[1:]))

In [14]:
data = data.drop(['price', 'weekly_price', 'monthly_price'], axis = 1)

In [15]:
data['day_price'] = day_price
data['week_price'] = week_price
data['month_price'] = month_price

## New variable : monthly price per square foot

In [16]:
price_sqft = []
for i in range(nrow):
    if pd.isnull(data.square_feet[i]) == False and data.square_feet[i] != 0:
        price_sqft.append(round(data.month_price[i]/data.square_feet[i], 2))
    else :
        price_sqft.append(None)

In [17]:
data['price_sqft'] = price_sqft

## Aggregate neighbourhoods

In [18]:
len(set(data.neighbourhood_cleansed)) 
# 186 different values with duplicates sometimes

186

There are too many different names of neighborhoods. Sometimes, two names represent the same district. We rename them using the zipcodes. This website gives the correspondence between zipcodes, districts and neighborhoods. https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm

We verify that there are not many missing values in the zipcode column less than 1%

In [19]:
# list of indices where zipcode is missing
non_missing_zip = [ind for ind in range(nrow) if pd.isnull(data.zipcode)[ind] == False] 
# get the proportion
len(non_missing_zip)/nrow 
# returns 0.994

0.9940858644859814

We can remove these missing values

In [20]:
data = data.ix[non_missing_zip,]
# reset indices
data = data.reset_index()

In [21]:
# update new number rows
nrow = len(data.index)

### Clean borough names, neighborhood names

In [22]:
borough_table = list()
neighborhood_table = list()
for i in range(nrow):
    if data.zipcode[i] in zip_code_unique:
        borough_table.append(borough[zip_code_unique.index(data.zipcode[i])])
        neighborhood_table.append(neighborhood[zip_code_unique.index(data.zipcode[i])])
    else:
        borough_table.append(" N/A ")
        neighborhood_table.append(" N/A ")
    
#print(borough_table)

In [23]:
data['borough'] = borough_table
data['district'] = neighborhood_table

In [25]:
print(len(data.ix[data.borough == " N/A "]))
print(len(data))
print(len(set(data.ix[data.borough == " N/A "])))

476
27230
22


In [28]:
df_all = data.ix[[ind for ind in range(nrow) if data.borough[ind] != " N/A "],]

print(len(df_all))
df_all.head(2)

26754


,index,name,neighbourhood_cleansed,city,zipcode,latitude,longitude,is_location_exact,property_type,room_type,...,bedrooms,beds,square_feet,review_scores_value,day_price,week_price,month_price,price_sqft,borough,district
0,0,Stay like a real New Yorker!,Midtown East,New York,10022,40.756852,-73.964754,t,Apartment,Entire home/apt,...,1,1,NaN,9,160,1000,4800,NaN,Manhattan,Gramercy Park and Murray Hill
1,2,Option of 2 Beds w Private Bathroom,Bushwick,Brooklyn,11221,40.692189,-73.924120,t,Apartment,Private room,...,1,2,NaN,10,58,406,1740,NaN,Brooklyn,Bushwick and Williamsburg


In [ ]:
df_all = df_all.drop(['index'], axis = 1)

# Export data

In [45]:
df_all.to_csv("cleaned_housing_data_all.csv")